In [1]:
# breakhistf2v26.ipynb  v1.0.0 
# 
#  --------------------------------------------------
#  Hangzhou Domain Zones Technology Co., Ltd

#  Apache Licence 2.0       https://www.apache.org/licenses/LICENSE-2.0
#  --------------------------------------------------


import os
import tensorflow as tf
import numpy as np
import csv
import matplotlib.pyplot as plt
from tensorflow import keras
import neural_structured_learning as nsl
from tensorflow.python.keras.api._v2.keras import layers, optimizers, losses
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import TensorBoard

tf.random.set_seed(22)
np.random.seed(22)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
assert tf.__version__.startswith('2.')
  
%matplotlib inline

def read_csv(csvnamepath, filename):
     

    # read from csv file
    images, labels = [], []
    with open(os.path.join(csvnamepath, filename)) as f:
        reader = csv.reader(f)
        for row in reader:
            img, label = row
            label = int(label)

            images.append(img)
            labels.append(label)

    assert len(images) == len(labels)

    return images, labels

 
def load_breakhis(dirc, mode='train'):
    classdir2label={}
    filename="tf2breakhisCSV"
    filedirs = os.listdir( dirc)
    for filedir in filedirs:
        if not os.path.isdir(os.path.join(dirc,filedir)):
            continue
        classdir2label[filedir]=len(classdir2label.keys())
    

     
    images, labels = read_csv(os.path.join(os.path.abspath('.'),'tf2breakhis'), filename )  
    if mode == 'train':  # 60%
        images = images[:int(0.6 * len(images))]
        labels = labels[:int(0.6 * len(labels))]
    elif mode == 'val':  # 20% = 60%->80%
        images = images[int(0.6 * len(images)):int(0.8 * len(images))]
        labels = labels[int(0.6 * len(labels)):int(0.8 * len(labels))]
    else:  # 20% = 80%->100%
        images = images[int(0.8 * len(images)):]
        labels = labels[int(0.8 * len(labels)):]
    return images, labels, classdir2label
 
img_mean = tf.constant([0.485, 0.456, 0.406])
img_std = tf.constant([0.229, 0.224, 0.225])

@tf.function
def normalize(x, mean=img_mean, std=img_std):
    # x shape: [224, 224, 3]
    # mean：shape为1；这里用到了广播机制。我们安装好右边对齐的原则，可以得到如下；
    # mean : [1, 1, 3], std: [3]        先插入1
    # mean : [224, 224, 3], std: [3]    再变为224
    x = (x - mean)/std
    return x

# 数据normalize之后，这里有一个反normalizaion操作。比如数据可视化的时候，需要反过来。
@tf.function
def denormalize(x, mean=img_mean, std=img_std):
    x = x * std + mean
    return x

 


# 预处理的函数，复制过来。
@tf.function
def preprocess(x,y):
    # x: 图片的路径，y：图片的数字编码
   
    x = tf.io.read_file(x)
     
    x = tf.image.decode_jpeg(x, channels=3) # RGBA
    
    x = tf.image.resize(x, [244, 244])
   
   
    x = tf.image.random_flip_left_right(x)
    #plt.imshow(x)
  
    # x = tf.image.random_flip_up_down(x)
    x = tf.image.random_crop(x, [224,224,3])
    #plt.imshow(x)
    
    # x: [0,255]=> -1~1
    x = tf.cast(x, dtype=tf.float32) / 255.
    x = normalize(x)
    y = tf.convert_to_tensor(y)
    y = tf.one_hot(y, depth=8)

    return x, y


# 预处理的函数，复制过来。
@tf.function
def preprocess1(x,y):
    # x: 图片的路径，y：图片的数字编码
   
    x = tf.io.read_file(x)
     
    x = tf.image.decode_jpeg(x, channels=3) # RGBA
    
    x = tf.image.resize(x, [244, 244])
   
    
    # x = tf.image.random_flip_up_down(x)
    x = tf.image.random_crop(x, [224,224,3])
      
    # x: [0,255]=> -1~1
    x = tf.cast(x, dtype=tf.float32) / 255.
    x = normalize(x)
    y = tf.convert_to_tensor(y)
    y = tf.one_hot(y, depth=8)

    return x, y




 

batchsz = 32
epochnum =10
# creat train db   一般训练的时候需要shuffle。其它是不需要的。
images, labels, table = load_breakhis(os.path.join(os.path.abspath('.'),'tf2breakhis'), 'train')
db_train = tf.data.Dataset.from_tensor_slices((images, labels))  # 变成个Dataset对象。
db_train = db_train.shuffle(1000).repeat(2).map(preprocess).shuffle(1000).batch(batchsz)  # map函数图片路径变为内容。
 

# crate validation db
images2, labels2, table = load_breakhis(os.path.join(os.path.abspath('.'),'tf2breakhis'), 'val')
db_val = tf.data.Dataset.from_tensor_slices((images2, labels2))
db_val = db_val.shuffle(1000).repeat(2).map(preprocess).shuffle(1000).batch(batchsz)
# create test db
images3, labels3, table = load_breakhis(os.path.join(os.path.abspath('.'),'tf2breakhis'), 'test')
db_test = tf.data.Dataset.from_tensor_slices((images3, labels3))
db_test = db_test.map(preprocess1).batch(batchsz)

 
resnetdense = keras.Sequential([
    layers.Conv2D(16,5,3),
    layers.MaxPool2D(3,3),
    layers.ReLU(),
    layers.Conv2D(64,5,3),
    layers.MaxPool2D(2,2),
    layers.ReLU(),
    
    layers.Flatten(),
    #
    layers.Dense(128), 
    layers.Dropout(rate=0.5),
    
    
    
    layers.Dense(64),  
    layers.ReLU(),
    layers.Dense(8)
   ])


 

# 首先创建Resnet18
# resnet = ResNet(8)

resnetdense.build(input_shape=(batchsz, 224, 224, 3))
resnetdense.summary()

# monitor监听器, 连续5个验证准确率不增加，这个事情触发。
 
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    min_delta=0.001,
    patience=100

)

  

 
# 网络的装配。
resnetdense.compile(optimizer=optimizers.Adam(lr=1e-4),
               loss=losses.CategoricalCrossentropy(from_logits=True),
               metrics=['accuracy'])

# 完成标准的train，val, test;
# 标准的逻辑必须通过db_val挑选模型的参数，就需要提供一个earlystopping技术，

LOGDIR='log/breakhistf2v26' 
resnetdense.fit(db_train, validation_data=db_val, 
                        validation_freq=10, epochs=2000, 
                        callbacks=[TensorBoard(log_dir=LOGDIR)])   # 1个epoch验证1次。触发了这个事情，提前停止了。
resnetdense.evaluate(db_test)
 
 

    
    
 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  1216      
_________________________________________________________________
max_pooling2d (MaxPooling2D) multiple                  0         
_________________________________________________________________
re_lu (ReLU)                 multiple                  0         
_________________________________________________________________
conv2d_1 (Conv2D)            multiple                  25664     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 multiple                  0         
_________________________________________________________________
re_lu_1 (ReLU)               multiple                  0         
_________________________________________________________________
flatten (Flatten)            multiple                  0

297/297 [==============================] - 299s 1s/step - loss: 0.5316 - accuracy: 0.7963 - val_loss: 0.7044 - val_accuracy: 0.7336
Epoch 131/2000
297/297 [==============================] - 246s 828ms/step - loss: 0.5327 - accuracy: 0.7941
Epoch 132/2000
297/297 [==============================] - 244s 822ms/step - loss: 0.5265 - accuracy: 0.7983
Epoch 133/2000
297/297 [==============================] - 243s 819ms/step - loss: 0.5158 - accuracy: 0.7994
Epoch 134/2000
297/297 [==============================] - 244s 823ms/step - loss: 0.5240 - accuracy: 0.8008
Epoch 135/2000
297/297 [==============================] - 245s 824ms/step - loss: 0.5296 - accuracy: 0.7982
Epoch 136/2000
297/297 [==============================] - 245s 827ms/step - loss: 0.5164 - accuracy: 0.8008
Epoch 137/2000
297/297 [==============================] - 243s 818ms/step - loss: 0.5173 - accuracy: 0.8000
Epoch 138/2000
297/297 [==============================] - 242s 815ms/step - loss: 0.5102 - accuracy: 0.8013
Epoc

297/297 [==============================] - 213s 718ms/step - loss: 0.3479 - accuracy: 0.8636
Epoch 276/2000
297/297 [==============================] - 211s 709ms/step - loss: 0.3348 - accuracy: 0.8674
Epoch 277/2000
297/297 [==============================] - 211s 711ms/step - loss: 0.3428 - accuracy: 0.8641
Epoch 278/2000
297/297 [==============================] - 210s 708ms/step - loss: 0.3304 - accuracy: 0.8663
Epoch 279/2000
297/297 [==============================] - 214s 721ms/step - loss: 0.3517 - accuracy: 0.8609
Epoch 280/2000
297/297 [==============================] - 279s 939ms/step - loss: 0.3324 - accuracy: 0.8690 - val_loss: 0.6410 - val_accuracy: 0.7835
Epoch 281/2000
297/297 [==============================] - 211s 710ms/step - loss: 0.3404 - accuracy: 0.8659
Epoch 282/2000
297/297 [==============================] - 222s 748ms/step - loss: 0.3341 - accuracy: 0.8655
Epoch 283/2000
297/297 [==============================] - 212s 714ms/step - loss: 0.3293 - accuracy: 0.8724
E

Epoch 421/2000
297/297 [==============================] - 243s 819ms/step - loss: 0.2694 - accuracy: 0.8972s - loss: 0.268
Epoch 422/2000
297/297 [==============================] - 241s 812ms/step - loss: 0.2466 - accuracy: 0.9016
Epoch 423/2000
297/297 [==============================] - 240s 806ms/step - loss: 0.2434 - accuracy: 0.9063
Epoch 424/2000
297/297 [==============================] - 274s 922ms/step - loss: 0.2570 - accuracy: 0.8950
Epoch 425/2000
297/297 [==============================] - 502s 2s/step - loss: 0.2668 - accuracy: 0.8955
Epoch 426/2000
297/297 [==============================] - 2756s 9s/step - loss: 0.2557 - accuracy: 0.8967
Epoch 427/2000
297/297 [==============================] - 220s 741ms/step - loss: 0.2411 - accuracy: 0.9014
Epoch 428/2000
297/297 [==============================] - 235s 792ms/step - loss: 0.2599 - accuracy: 0.8964
Epoch 429/2000
297/297 [==============================] - 225s 756ms/step - loss: 0.2428 - accuracy: 0.9017
Epoch 430/2000
297

297/297 [==============================] - 212s 714ms/step - loss: 0.2212 - accuracy: 0.9131
Epoch 495/2000
297/297 [==============================] - 212s 714ms/step - loss: 0.2164 - accuracy: 0.9134
Epoch 496/2000
297/297 [==============================] - 214s 719ms/step - loss: 0.2338 - accuracy: 0.9072
Epoch 497/2000
297/297 [==============================] - 214s 720ms/step - loss: 0.2235 - accuracy: 0.9120
Epoch 498/2000
297/297 [==============================] - 213s 718ms/step - loss: 0.2191 - accuracy: 0.9103
Epoch 499/2000
297/297 [==============================] - 215s 725ms/step - loss: 0.2143 - accuracy: 0.9171
Epoch 500/2000
297/297 [==============================] - 283s 952ms/step - loss: 0.2279 - accuracy: 0.9115 - val_loss: 0.6929 - val_accuracy: 0.7924
Epoch 501/2000
297/297 [==============================] - 214s 719ms/step - loss: 0.2125 - accuracy: 0.9138
Epoch 502/2000
297/297 [==============================] - 213s 718ms/step - loss: 0.2151 - accuracy: 0.9159
E

...

297/297 [==============================] - 218s 732ms/step - loss: 0.1109 - accuracy: 0.9565
Epoch 1509/2000
297/297 [==============================] - 199s 670ms/step - loss: 0.1150 - accuracy: 0.9547
Epoch 1510/2000
297/297 [==============================] - 261s 877ms/step - loss: 0.1028 - accuracy: 0.9573 - val_loss: 0.8801 - val_accuracy: 0.8101
Epoch 1511/2000
297/297 [==============================] - 194s 654ms/step - loss: 0.1052 - accuracy: 0.9588
Epoch 1512/2000
297/297 [==============================] - 196s 661ms/step - loss: 0.1040 - accuracy: 0.9580
Epoch 1513/2000
297/297 [==============================] - 194s 654ms/step - loss: 0.0920 - accuracy: 0.9633
Epoch 1514/2000
297/297 [==============================] - 194s 653ms/step - loss: 0.1052 - accuracy: 0.9585
Epoch 1515/2000
297/297 [==============================] - 201s 675ms/step - loss: 0.1024 - accuracy: 0.9601
Epoch 1516/2000
297/297 [==============================] - 197s 664ms/step - loss: 0.1057 - accuracy: 

297/297 [==============================] - 204s 687ms/step - loss: 0.0983 - accuracy: 0.9605
Epoch 1653/2000
297/297 [==============================] - 203s 682ms/step - loss: 0.0911 - accuracy: 0.9634
Epoch 1654/2000
297/297 [==============================] - 201s 678ms/step - loss: 0.0910 - accuracy: 0.9642
Epoch 1655/2000
297/297 [==============================] - 204s 688ms/step - loss: 0.0968 - accuracy: 0.9617
Epoch 1656/2000
297/297 [==============================] - 201s 677ms/step - loss: 0.0957 - accuracy: 0.9619
Epoch 1657/2000
297/297 [==============================] - 204s 686ms/step - loss: 0.1046 - accuracy: 0.9605
Epoch 1658/2000
297/297 [==============================] - 203s 685ms/step - loss: 0.0980 - accuracy: 0.9634
Epoch 1659/2000
297/297 [==============================] - 203s 682ms/step - loss: 0.1017 - accuracy: 0.9603
Epoch 1660/2000
297/297 [==============================] - 263s 887ms/step - loss: 0.0865 - accuracy: 0.9652 - val_loss: 0.8390 - val_accuracy: 

297/297 [==============================] - 191s 642ms/step - loss: 0.0859 - accuracy: 0.9651
Epoch 1797/2000
297/297 [==============================] - 192s 646ms/step - loss: 0.0951 - accuracy: 0.9631
Epoch 1798/2000
297/297 [==============================] - 190s 639ms/step - loss: 0.0881 - accuracy: 0.9651
Epoch 1799/2000
297/297 [==============================] - 189s 635ms/step - loss: 0.0895 - accuracy: 0.9642
Epoch 1800/2000
297/297 [==============================] - 250s 840ms/step - loss: 0.0957 - accuracy: 0.9616 - val_loss: 0.8580 - val_accuracy: 0.8142
Epoch 1801/2000
297/297 [==============================] - 191s 644ms/step - loss: 0.0988 - accuracy: 0.9606
Epoch 1802/2000
297/297 [==============================] - 190s 640ms/step - loss: 0.0952 - accuracy: 0.9626
Epoch 1803/2000
297/297 [==============================] - 189s 637ms/step - loss: 0.0847 - accuracy: 0.9673
Epoch 1804/2000
297/297 [==============================] - 192s 646ms/step - loss: 0.0905 - accuracy: 

297/297 [==============================] - 267s 898ms/step - loss: 0.0865 - accuracy: 0.9634 - val_loss: 0.9204 - val_accuracy: 0.8097
Epoch 1941/2000
297/297 [==============================] - 205s 689ms/step - loss: 0.0867 - accuracy: 0.9663
Epoch 1942/2000
297/297 [==============================] - 203s 683ms/step - loss: 0.0906 - accuracy: 0.9630
Epoch 1943/2000
297/297 [==============================] - 205s 690ms/step - loss: 0.0882 - accuracy: 0.9648
Epoch 1944/2000
297/297 [==============================] - 205s 690ms/step - loss: 0.0890 - accuracy: 0.9627
Epoch 1945/2000
297/297 [==============================] - 206s 694ms/step - loss: 0.0818 - accuracy: 0.9685
Epoch 1946/2000
297/297 [==============================] - 206s 693ms/step - loss: 0.0906 - accuracy: 0.9669
Epoch 1947/2000
297/297 [==============================] - 203s 684ms/step - loss: 0.0913 - accuracy: 0.9639
Epoch 1948/2000
297/297 [==============================] - 207s 697ms/step - loss: 0.0890 - accuracy: 

[1.0118081665039063, 0.8223767]